In [34]:
#Copula approach
    #Step 1: filter pairs based on kendall tau
    #step 2: find best fit distribution
    #Step 3: find best fit copula
    #Step 4: use copula formula to calculate conditional probability and create trading strategy
    #Step 5: get results
#edits:
    #added rolling window to cond prob calculation to avoid future bias
    #added position shift like other approach

import pandas as pd
import numpy as np
from scipy import stats
from copulae import GaussianCopula, ClaytonCopula, GumbelCopula
import warnings
import yfinance as yf
import matplotlib.pyplot as plt
warnings.filterwarnings("ignore")


# Select a wide range of assets you're interested in
assets = ['ES=F', 'CL=F', 'GC=F', 'SI=F', 'AAPL', 'MSFT', 'GOOGL', 'AMZN', 
          'JPM', 'V', 'PG', 'XOM', 'GLD', 'SLV', 'USO', 'SPY', 'TLT', 'QQQ', 'DIA', 
          'NFLX', 'TSLA', 'BA', 'KO', 'DIS', 'PLUG', 'GS', 'IBM', 'INTC', 'ORCL', 'NVDA']

start_date = '2000-01-01'
end_date = '2021-12-31'

# Fetch the data for all assets
asset_data = []
asset_dict = {}
for i,  asset in enumerate(assets):
    try:
        df = yf.download(asset, start=start_date, end=end_date)
        if not df.empty:  # If df is not empty
            df = df.resample('B').mean().fillna(method='ffill')  # Resample at daily frequency
            asset_data.append(df['Close'])
            asset_dict[asset] = i
    except Exception as e:
        print(f"Failed to download {asset}: {str(e)}")


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

In [35]:
#Apply testing period for assets
test_start = pd.Timestamp("2000-01-01")
test_end = pd.Timestamp("2019-12-31")

#Step 1: Select pairs with highest kendall tau
    #A statistic used to measure the ordinal association between two measured quantities
selected_pairs = []
# Perform analysis on each asset pair and select the pairs
for i in range(len(asset_data)):
    for j in range(i+1, len(asset_data)):
        asset1 = asset_data[i].loc[test_start:test_end]
        asset2 = asset_data[j].loc[test_start:test_end]
        prices1 = asset1
        prices2 = asset2
        returns1 = np.log(prices1).diff().dropna()
        returns2 = np.log(prices2).diff().dropna()
       
        # Ensure returns1 and returns2 have the same length
        min_len = min(len(returns1), len(returns2))
        returns1 = returns1[:min_len]
        returns2 = returns2[:min_len]
   
        # Align the two return series on their index: due to missing data
        aligned_returns1, aligned_returns2 = returns1.align(returns2, join='inner')
        ktcorr = stats.kendalltau(aligned_returns1, aligned_returns2)[0]


        #filter pairs based on ktcorr>0.35
        if ktcorr > 0.35:
            selected_pairs.append((assets[i], assets[j], ktcorr))

selected_pairs = selected_pairs[:20]

# Sort the filtered pairs based on ktcorr in ascending order
selected_pairs.sort(key=lambda x: x[2], reverse=True)
for i, pair in enumerate(selected_pairs):
    print(f"Pair {i+1}: {pair[0]} and {pair[1]} with ktcorr {pair[2]}")

Pair 1: ES=F and SPY with ktcorr 0.8898432086383968
Pair 2: CL=F and USO with ktcorr 0.8253174375037056
Pair 3: ES=F and DIA with ktcorr 0.787541014847115
Pair 4: GC=F and GLD with ktcorr 0.7441352113386912
Pair 5: ES=F and QQQ with ktcorr 0.6674910264566074
Pair 6: GC=F and SI=F with ktcorr 0.586583621182953
Pair 7: ES=F and JPM with ktcorr 0.5313047901130323
Pair 8: ES=F and GS with ktcorr 0.5115093273965535
Pair 9: ES=F and MSFT with ktcorr 0.47730884944224294
Pair 10: ES=F and DIS with ktcorr 0.47522514227334767
Pair 11: ES=F and IBM with ktcorr 0.4741621621302099
Pair 12: ES=F and INTC with ktcorr 0.4661730014111685
Pair 13: ES=F and ORCL with ktcorr 0.45427688270914895
Pair 14: ES=F and XOM with ktcorr 0.44878829544263743
Pair 15: ES=F and BA with ktcorr 0.42623490422497323
Pair 16: ES=F and V with ktcorr 0.4189230727819263
Pair 17: ES=F and GOOGL with ktcorr 0.39828410872723097
Pair 18: ES=F and AMZN with ktcorr 0.38625532970828885
Pair 19: ES=F and AAPL with ktcorr 0.3839409333

In [36]:
#Step 2: Select marginal distribution using the Akaike Information Criterion (AIC) and filter based on those with student t
    #AIC is used to compare different possible models and determine which one is the best fit for the data
    #is calculated from
        #the number of independent variables used to build the model
        #the maximum likelihood estimate of the model
def compute_best_dist(data):
    dists = ['Normal', "Student's t", 'Logistic', 'Extreme']
    best_aic = np.inf #compare and seek the min AIC value
    for dist, name in zip([stats.norm, stats.t, stats.genlogistic, stats.genextreme], dists):
        params = dist.fit(data)
        dist_fit = dist(*params)
        log_like = np.log(dist_fit.pdf(data)).sum()
        aic = 2*len(params) - 2 * log_like
        if aic<best_aic:
            best_dist = name
            best_aic = aic
    return [best_dist]


# Dictionary to store the best fitting distribution for each asset
asset_dist_dict = {}
for asset1, asset2, ktcorr in selected_pairs:

    # Calculate the best fitting distribution for each asset's returns
    asset_dist_dict[asset1] = compute_best_dist(returns1)
    asset_dist_dict[asset2] = compute_best_dist(returns2)
     # Check if both assets' returns best fit to Student's t distribution
    if compute_best_dist(returns1) == "Normal" and compute_best_dist(returns2) == "Normal":
        selected_pairs.append((asset1, asset2, ktcorr))

In [37]:
#trading period
trade_start = pd.Timestamp("2020-01-01")
trade_end = pd.Timestamp("2020-12-31")
# Initialize empty lists for storing calculated values
portfolio_returns = []
portfolio_volatility = []
for asset1, asset2, ktcorr in selected_pairs:    # Get the returns of the testing period
    # Get the returns of the trading period
    asset1_trade = asset_data[asset_dict[asset1]].loc[start_date:end_date]
    asset2_trade = asset_data[asset_dict[asset2]].loc[start_date:end_date]
    prices1_trade = asset1_trade
    prices2_trade = asset2_trade
    returns1_trade = np.log(prices1_trade).diff().dropna()
    returns2_trade = np.log(prices2_trade).diff().dropna()
    # Ensure returns1_trade and returns2_trade have the same length
    min_len = min(len(returns1_trade), len(returns2_trade))
    returns1_trade = returns1_trade[:min_len]
    returns2_trade = returns2_trade[:min_len]
   
    # fit marginals
    params_s1 = stats.t.fit(returns1)
    dist_s1 = stats.t(*params_s1)
    params_s2 = stats.t.fit(returns2)
    dist_s2 = stats.t(*params_s2)
   
    # transform marginals
    u = dist_s1.cdf(returns1)
    v = dist_s2.cdf(returns2)
    min_len = min(len(u), len(v))
    u = u[:min_len]
    v = v[:min_len]
    data = np.column_stack([u, v])


    #Step 3: Select best fitting copula
    best_aic = np.inf
    copulas = [GaussianCopula(dim=2), ClaytonCopula(dim=2), GumbelCopula(dim=2)]
    for copula in copulas:
        copula.fit(data)
        L = copula.log_lik(data)
        aic = 2 * 1 - 2 * L  # copula models have 1 parameter
        if aic < best_aic:
            best_aic = aic
            best_copula = copula
            
    copula_name = str(type(best_copula)).split('.')[-1].replace("'>", '')
    print(f'{asset1} and {asset2}: {copula_name}')
        # Since all pairs best copula is Gumbel can proceed using its formulae 
    #Step 4: calculate conditional probability and create trading positions        
    # Create positions for trading period
    positions1 = []
    positions2 = []
    u = returns1_trade.apply(dist_s1.cdf).rolling(30).mean().dropna()
    v = returns2_trade.apply(dist_s2.cdf).rolling(30).mean().dropna()
    # Getting the min length from u, v and returns to align them
    min_len = min(len(u), len(v), len(returns1_trade), len(returns2_trade))
    u = u[:min_len]
    v = v[:min_len]
    returns1_trade = returns1_trade[:min_len]
    returns2_trade = returns2_trade[:min_len]

    #Best copula model for all pairs is Gumbel, so calculate conditional probability based on gumbel copula formula
    #Conditional probability u given v (C(u|v) for Gumbel copula)
        #Formula: C(u|v; θ) = exp(-((-log(v))^θ + (-log(u))^θ)^(1/θ) / v)
            #theta is the copula parameter
    theta = best_copula.params  # copula parameter
    # Conditional probability u given v for Gumbel copula
    prob_u_given_v = np.exp(-((-np.log(v))**theta + (-np.log(u))**theta)**(1/theta)) / v
    # Conditional probability v given u for Gumbel copula
    prob_v_given_u = np.exp(-((-np.log(u))**theta + (-np.log(v))**theta)**(1/theta)) / u
    threshold = 0.50

    # Create positions based on the conditional probabilities
    positions1 = [1 if p > threshold else -1 for p in prob_u_given_v]
    positions2 = [1 if p > threshold else -1 for p in prob_v_given_u]
    # Getting the min length from positions1, positions2 and returns to align them
    min_len = min(len(positions1), len(positions2), len(returns1_trade), len(returns2_trade))
    positions1 = positions1[:min_len]
    positions2 = positions2[:min_len]
    returns1_trade = returns1_trade[:min_len]
    returns2_trade = returns2_trade[:min_len]
    positions1 = pd.Series(positions1, index=returns1_trade.index)
    positions2 = pd.Series(positions2, index=returns2_trade.index)
    returns1_from_positions = returns1_trade * positions1.shift(2)
    returns2_from_positions = returns2_trade * positions2.shift(2)

    # Combine returns of the two assets
    total_returns = returns1_from_positions + returns2_from_positions
    # Calculate the average return
    total_return_mean = total_returns.mean()
    # Calculate the standard deviation of the returns
    std_dev = total_returns.std()
    portfolio_returns.append(total_return_mean)
    portfolio_volatility.append(std_dev)

ES=F and SPY: GumbelCopula
CL=F and USO: GumbelCopula
ES=F and DIA: GumbelCopula
GC=F and GLD: GumbelCopula
ES=F and QQQ: GumbelCopula
GC=F and SI=F: GumbelCopula
ES=F and JPM: GumbelCopula
ES=F and GS: GumbelCopula
ES=F and MSFT: GumbelCopula
ES=F and DIS: GumbelCopula
ES=F and IBM: GumbelCopula
ES=F and INTC: GumbelCopula
ES=F and ORCL: GumbelCopula
ES=F and XOM: GumbelCopula
ES=F and BA: GumbelCopula
ES=F and V: GumbelCopula
ES=F and GOOGL: GumbelCopula
ES=F and AMZN: GumbelCopula
ES=F and AAPL: GumbelCopula
ES=F and NVDA: GumbelCopula


In [38]:
#Step 5: get results
for i, (asset1, asset2, ktcorr) in enumerate(selected_pairs):
    sharpe_ratio = np.sqrt(252) * portfolio_returns[i] / (portfolio_volatility[i])
    print(f"Pair {i+1}: {asset1}, {asset2}, Sharpe Ratio: {sharpe_ratio}, Copula: {copula_name}, Kendall Tau correlation: {ktcorr}")

Pair 1: ES=F, SPY, Sharpe Ratio: 0.2771908347015686, Copula: GumbelCopula, Kendall Tau correlation: 0.8898432086383968
Pair 2: CL=F, USO, Sharpe Ratio: -0.23692609867369413, Copula: GumbelCopula, Kendall Tau correlation: 0.8253174375037056
Pair 3: ES=F, DIA, Sharpe Ratio: 0.295866348008755, Copula: GumbelCopula, Kendall Tau correlation: 0.787541014847115
Pair 4: GC=F, GLD, Sharpe Ratio: 0.19466305790360694, Copula: GumbelCopula, Kendall Tau correlation: 0.7441352113386912
Pair 5: ES=F, QQQ, Sharpe Ratio: 0.2761208694208075, Copula: GumbelCopula, Kendall Tau correlation: 0.6674910264566074
Pair 6: GC=F, SI=F, Sharpe Ratio: 0.3544500023238844, Copula: GumbelCopula, Kendall Tau correlation: 0.586583621182953
Pair 7: ES=F, JPM, Sharpe Ratio: 0.22619012627443316, Copula: GumbelCopula, Kendall Tau correlation: 0.5313047901130323
Pair 8: ES=F, GS, Sharpe Ratio: 0.2154038549166779, Copula: GumbelCopula, Kendall Tau correlation: 0.5115093273965535
Pair 9: ES=F, MSFT, Sharpe Ratio: 0.34539040752

In [39]:
# Best perfoming pair is ES=F and AAPL with sharpe ratio of 0.554
# Limitations
    # Only relied on pairs with Normal distribution
    # Limited Asset Pair Selection: The code only filters asset pairs based on the Kendall Tau correlation above 0.35
    # Lack of Transaction Cost Consideration: The code assumes no transaction costs
    # Lack of Risk Management: The code doesn't consider drawdown or risk-adjusted returns